In [3]:
from flair.data import Sentence
from flair.nn import Classifier

# make a sentence
sentence = Sentence('George Washington went to Washington.')

# load the NER tagger
tagger = Classifier.load('ner-large')

# run NER over sentence
tagger.predict(sentence)

# print the sentence with all annotations
print(sentence)

2024-01-25 09:12:35,114 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence[6]: "George Washington went to Washington." → ["George Washington"/PER, "Washington"/LOC]


In [32]:
from docx import Document
import re
doc = Document('C:\\Users\\hechu\\Downloads\\1st_Meeting_3rd_Session_Open-ended_Working_Group_on_Reducing_Space_Threats.docx')
speeches = {}

def clean(text):
    # Remove timestamps
    clean_text = re.sub(r'\d{2}:\d{2}:\d{2}\s*', '', text)
    return clean_text


paragraphs = [clean(para.text) for para in doc.paragraphs if para.text.strip() != '']


# Define the path for the output text file
output_file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\output.txt'

# Write paragraphs to the text file
with open(output_file_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write('\n'.join(paragraphs))

txt_file.close()

In [35]:
# Read the text file
file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\output.txt'  # Update with your file path

with open(file_path, 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()

# Dictionary to store speeches
speeches = {}
current_speaker = None
previous_speaker = None

# Process each line
for line in lines:
    if re.match(r'Speaker \d+', line.strip()):
        previous_speaker = current_speaker
        current_speaker = line.strip()
        if current_speaker not in speeches:
            speeches[current_speaker] = ''
    else:
        speeches[current_speaker] += line.strip() + ' '

txt_file.close()
# Write the consolidated speeches back to a file
output_file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\consolidated_output.txt'  # Update with your desired output path
with open(output_file_path, 'w', encoding='utf-8') as file:
    for speaker, speech in speeches.items():
        file.write(f"{speaker}\n{speech}\n\n")
file.close()

In [39]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import spacy

with open('C:\\Users\\hechu\\Desktop\\AICP\\consolidated_output.txt', 'r', encoding='utf-8') as file:
    text = file.read()

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
sentences = [sent.text.strip() for sent in doc.sents]


key = "4011e47207ed418b9196929c20a002bd"
endpoint = "https://eastus.api.cognitive.microsoft.com/"

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example function for recognizing entities from text
def entity_recognition(client):
    try:
        documents = [{"id": str(idx), "text": sent} for idx, sent in enumerate(sentences)]
        result = client.recognize_entities(documents = documents)

        if doc in result:
            if not doc.is_error:
                print("Named Entities:\n")
                for entity in doc.entities:
                    print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                            "\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")
            else:
                print(f"Error: {doc.error}")
    except Exception as err:
        print("Encountered exception. {}".format(err))
entity_recognition(client)


2024-01-25 12:47:02,011 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [41]:
with open('C:\\Users\\hechu\\Desktop\\AICP\\annotated_output.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(sentence.to_tagged_string())